look at work on Randstad laptop

In [3]:
import pandas as pd
import numpy as np
import pickle
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 

In [6]:
with open('../../../data/iplan_column_names.pkl', 'rb') as f:
    column_names = pickle.load(f)

#with open('../data/iplan_date_columns.pkl', 'rb') as f:
#    date_columns = pickle.load(f)
date_columns = ['accept_by_employee_date_time']

data_folder = '../../../data/'
data_file = 'repo_iplan_employee_in_shift_20200210-20200216.csv'

In [2]:
usecols = ['shift_id','accept_by_employee_date_time','pool_id']

In [7]:
df = pd.read_csv(data_folder+data_file,names=column_names, parse_dates=date_columns) #,usecols=usecols)

/Users/benplatten/.pyenv/versions/werkcollege/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (19,53,67,68,69) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [16]:
df.shape

(248119, 84)

In [25]:
cols = df.columns.to_list()
cols.sort()
cols

['accept_by_cust_with_type_id',
 'accept_by_customer_date_time',
 'accept_by_emp_with_type_id',
 'accept_by_employee_date_time',
 'batch_ts',
 'batchid',
 'called_up_date_time',
 'called_up_with_type_id',
 'changed_event_date_time',
 'changed_planned_hours_dt',
 'clocked_end_date_time',
 'clocked_start_date_time',
 'comm_to_customer_date_time',
 'comm_to_customer_with_type_id',
 'comm_to_employee_date_time',
 'comm_to_employee_with_type_id',
 'confirmed_date_time',
 'confirmed_with_type_id',
 'creation_date_time',
 'creation_with_type_id',
 'current_call_up_id',
 'current_qual_percentage',
 'customer_in_pool_id',
 'decl_examined_date_time',
 'decl_to_erp_system_date_time',
 'decl_to_erp_system_tag',
 'decl_to_erp_with_type_id',
 'declared_end_date_time',
 'declared_event_id',
 'declared_start_date_time',
 'deleted_date_time',
 'deleted_with_type_id',
 'deviant_realization_end_dt',
 'deviant_realization_event_id',
 'deviant_realization_start_dt',
 'dms_timestamp',
 'employee_id',
 'end_

### What should the audience conclude from my EDA visualisations?

Range in complexity - small pools and very large pool  
But most are small  


In [ ]:
# smallest pool 
# largest pool

### Analysis: accepted, realised and confirmed

In [9]:
# accept_by_employee_date_time
# 

pooldf = df[['pool_id','shift_id','employee_id']].groupby(['pool_id']).agg({ 'shift_id' : 'nunique', 'employee_id' : 'nunique'})
pooldf.columns = ['num_shifts','num_employees']

In [12]:
print(len(pooldf[pooldf['num_shifts'] > 7]))
print(len(pooldf[pooldf['num_shifts'] <= 7]))

440
32


In [14]:
mediumPools = pooldf[(pooldf['num_shifts'] > 7) & (pooldf['num_shifts'] < 200)]
mediumPools.shape

(355, 2)

In [15]:
mediumPools.head()

,num_shifts,num_employees
pool_id,,
1,24,11
24,90,62
29,51,15
32,80,23
66,8,3
